# Climatology generator demo sandbox

Date: 13 Novebmer, 2023

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

### BRAN2020 is 16TB of data over nearly 9000 `netcdf` file assests in total.

#### required packages

In [1]:
import intake
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

#### start a local Dask client

In [2]:
from dask.distributed import Client
#client = Client(threads_per_worker=1)
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34783,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.20 GiB
Comm: tcp://127.0.0.1:41163,Total threads: 4
Dashboard: /proxy/41475/status,Memory: 35.89 GiB
Nanny: tcp://127.0.0.1:39503,


#### ignore warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')

# ARD workflow

### read paths from config file

#### [ you will need to specifiy your correct path for the `data-catalogue/config.ini` file ]

In [ ]:
import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Read the config file
#########
#### you will need to specifiy your correct path the the `data-catalogue/config.ini` file 
#########
config.read('./code/BRAN2020-intake-catalog/config.ini')

# Get the value of a variable
catalog_path = config.get('paths', 'catalog_path')

In [ ]:
catalog_path

In [ ]:
BRAN2020_catalog = intake.open_esm_datastore(catalog_path+'BRAN2020.json',columns_with_iterables=['variable'])

In [ ]:
BRAN2020_catalog

In [ ]:
BRAN2020_catalog.unique()['source']

In [ ]:
BRAN2020_catalog.unique()['domain']

In [ ]:
BRAN2020_catalog.unique()['time_period']

In [ ]:
var_list = BRAN2020_catalog.unique()['variable']
var_list.sort()
df = pd.DataFrame(var_list, columns=['BRAN2020 Variables'])
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df)

In [ ]:
search = BRAN2020_catalog.search(variable=['temp'],time_period='month')

In [ ]:
search.unique()['variable']

In [ ]:
search

In [ ]:
search.df

### Out of some 9000 files we now have narrowed down the just the 360 that have the information we need

### We can now load all that data into a single 396GB "lazy" `xarray` object for further reduction and analysis
#### Size is reduced from 16TB to 396GB

In [ ]:
%%time
DS = search.to_dask()

In [ ]:
DS

In [ ]:
DS.nbytes/1e9

## ARD - write zarr & chunk & write zarr

## 360 chunks is a bit too big for typical uses ( YMMV ) so rechunk depth by 10 = ~200MB

In [ ]:
DS_rc = DS.chunk({'st_ocean':10})
DS_rc

In [ ]:
BRAN2020_ard_path = '/scratch/es60/ard/reanalysis/BRAN2020/'
ard_file_ID = 'BRAN2020-monthly-temp-v13112023b.zarr'

In [ ]:
%%time
DS_rc.to_zarr(BRAN2020_ard_path+ard_file_ID,consolidated=True)
# ----- Client(threads_per_worker=1) -------
# CPU times: user 2min 26s, sys: 29.9 s, total: 2min 56s
# Wall time: 13min 1s
# ----- Client() -------
# CPU times: user 1min 53s, sys: 18.1 s, total: 2min 11s
# Wall time: 14min 3s

## RELOAD in ARD collection

In [ ]:
BRAN2020_monthly_temp = xr.open_zarr(BRAN2020_ard_path+ard_file_ID,consolidated=True)
# size of collection is 169GB with compression

In [ ]:
BRAN2020_monthly_temp

## CHUNK for time and WRITE zarr

### "Consider either rechunking using `chunk()`, deleting or modifying `encoding['chunks']`"

In [ ]:
def remove_zarr_encoding(DS):
    for var in DS:
        DS[var].encoding = {}

    for coord in DS.coords:
        DS[coord].encoding = {}
    return DS

In [ ]:
ard_rcTime_file_ID = 'BRAN2020-monthly-temp-chunk4time-v13112023.zarr'

In [ ]:
BRAN2020_monthly_temp_rcTime =  BRAN2020_monthly_temp.chunk({'Time':-1,'st_ocean':1,'xt_ocean':100})

In [ ]:
BRAN2020_monthly_temp_rcTime = remove_zarr_encoding(BRAN2020_monthly_temp_rcTime)

In [ ]:
%%time
BRAN2020_monthly_temp_rcTime.to_zarr(BRAN2020_ard_path+ard_rcTime_file_ID,consolidated=True)
#  ----- Client() -------
# CPU times: user 2min 55s, sys: 24.4 s, total: 3min 20s
# Wall time: 14min 10s
#

## end ARD workflow $\Uparrow$

# begin post-processing workflow $\Downarrow$

# load in both zarr collections

In [4]:
BRAN2020_ard_path = '/scratch/es60/ard/reanalysis/BRAN2020/'
ard_file_ID = 'BRAN2020-monthly-temp-v13112023.zarr'
ard_rcTime_file_ID = 'BRAN2020-monthly-temp-chunk4time-v13112023.zarr'

In [5]:
temp_chunked_time = xr.open_zarr(BRAN2020_ard_path + ard_rcTime_file_ID,consolidated=True)
temp_chunked = xr.open_zarr(BRAN2020_ard_path + ard_file_ID,consolidated=True)

# basic functions

In [6]:
def get_monthly_climatology(xr_object,time_coord_name = 'time',flox=True):
    if flox == True:
        monthly_climatology = xr_object.groupby(time_coord_name+'.month').mean(dim=time_coord_name,keep_attrs = True,method="cohorts", engine="flox")
    else:
        monthly_climatology = xr_object.groupby(time_coord_name+'.month').mean(dim=time_coord_name,keep_attrs = True)
    return monthly_climatology

def get_monthly_anomaly(xr_object,monthly_climatology, time_coord_name = 'time'):
    monthly_anomaly = xr_object.groupby(time_coord_name+'.month') - monthly_climatology
    return monthly_anomaly


# Climatology

In [7]:
temp_monthly_climatology = get_monthly_climatology(temp_chunked_time, time_coord_name = 'Time')
temp_monthly_climatology

<xarray.Dataset>
Dimensions:   (month: 12, st_ocean: 51, yt_ocean: 1500, xt_ocean: 3600)
Coordinates:
  * st_ocean  (st_ocean) float64 2.5 7.5 12.5 ... 2.895e+03 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 0.05 0.15 0.25 0.35 ... 359.6 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
  * month     (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    temp      (month, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(12, 1, 1500, 100), meta=np.ndarray>
Attributes: (12/13)
    NumFilesInSet:                   20
    acknowledgement:                 BRAN is made freely available by CSIRO B...
    catalogue_doi_url:               http://dx.doi.org/10.25914/6009627c7af03
    grid_tile:                       N/A
    grid_type:                       regular
    intake_esm_attrs:_data_format_:  netcdf
    ...                              ...
    intake_esm_attrs:source:         BRAN2020
    intake_esm_attrs:time_period:    month
    intake_esm_attrs:variable:       temp
    intake_esm_dataset_key:          ocean.month
    intake_esm_vars:                 ['temp']
    title:                           BRAN2020

In [8]:
temp_monthly_climatology_rc = temp_monthly_climatology.chunk({'st_ocean':10,'xt_ocean':3600,'month':1})

In [9]:
%%time
temp_monthly_climatology = temp_monthly_climatology.compute()

# CPU times: user 32.1 s, sys: 17.9 s, total: 49.9 s
# Wall time: 2min 20s
#

CPU times: user 28.6 s, sys: 18.1 s, total: 46.7 s
Wall time: 2min 14s


# Anomaly

In [10]:
temp_anomaly = get_monthly_anomaly(temp_chunked,temp_monthly_climatology_rc,time_coord_name='Time')

In [11]:
temp_anomaly

<xarray.Dataset>
Dimensions:   (Time: 360, st_ocean: 51, xt_ocean: 3600, yt_ocean: 1500)
Coordinates:
  * Time      (Time) datetime64[ns] 1993-01-16T12:00:00 ... 2022-12-16T12:00:00
  * st_ocean  (st_ocean) float64 2.5 7.5 12.5 ... 2.895e+03 3.603e+03 4.509e+03
  * xt_ocean  (xt_ocean) float64 0.05 0.15 0.25 0.35 ... 359.6 359.8 359.9 360.0
  * yt_ocean  (yt_ocean) float64 -74.95 -74.85 -74.75 ... 74.75 74.85 74.95
    month     (Time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12
Data variables:
    temp      (Time, st_ocean, yt_ocean, xt_ocean) float32 dask.array<chunksize=(1, 10, 1500, 3600), meta=np.ndarray>

In [ ]:
SST_anomaly = temp_anomaly.isel(st_ocean=0)
SST_anomaly

In [ ]:
SST_anomaly.temp.min('Time').plot()

# define El Nino and La Nina using NCAR ONI data

In [25]:
ONI_DF = pd.read_csv('/g/data/xv83/users/tm4888/data/ENSO/NCAR_ONI.csv')

In [26]:
ONI_DF.set_index('datetime',inplace=True)

In [27]:
ONI_DF

,year,month,ONI
datetime,,,
1950-01-15,1950,1,-1.53
1950-02-15,1950,2,-0.67
1950-03-15,1950,3,-0.43
1950-04-15,1950,4,-0.14
1950-05-15,1950,5,0.16
...,...,...,...
2023-08-15,2023,8,1.32
2023-09-15,2023,9,1.54
2023-10-15,2023,10,NaN


In [ ]:
EL_NINO = ONI_DF.where(ONI_DF['ONI'] >= 0.5)

In [ ]:
EL_NINO['ONI'].plot()

In [34]:
el_nino_threshold = 0.5
la_nina_threshold = -0.5

el_nino_threshold_months = ONI_DF["ONI"].ge(el_nino_threshold)
la_nina_threshold_months = ONI_DF["ONI"].le(la_nina_threshold)

In [32]:
el_nino_threshold_months[0:30]

datetime
1950-01-15    False
1950-02-15    False
1950-03-15    False
1950-04-15    False
1950-05-15    False
1950-06-15    False
1950-07-15     True
1950-08-15     True
1950-09-15     True
1950-10-15     True
1950-11-15    False
1950-12-15    False
1951-01-15    False
1951-02-15    False
1951-03-15    False
1951-04-15    False
1951-05-15    False
1951-06-15     True
1951-07-15     True
1951-08-15     True
1951-09-15     True
1951-10-15     True
1951-11-15     True
1951-12-15     True
1952-01-15     True
1952-02-15    False
1952-03-15    False
1952-04-15    False
1952-05-15    False
1952-06-15    False
Name: ONI, dtype: bool

In [33]:
ONI_DF = pd.concat([ONI_DF, el_nino_threshold_months.diff().ne(0).cumsum().rename('ENSO event group ID')], axis=1)
ONI_DF[0:30]

,year,month,ONI,ENSO event group ID
datetime,,,,
1950-01-15,1950,1,-1.53,1
1950-02-15,1950,2,-0.67,1
1950-03-15,1950,3,-0.43,1
1950-04-15,1950,4,-0.14,1
1950-05-15,1950,5,0.16,1
1950-06-15,1950,6,0.48,1
1950-07-15,1950,7,0.77,2
1950-08-15,1950,8,1.07,2
1950-09-15,1950,9,1.32,2


In [24]:
ONI_DF.groupby('ENSO event group ID').transform('count')

,year,month,ONI
datetime,,,
1950-01-15,6,6,6
1950-02-15,6,6,6
1950-03-15,6,6,6
1950-04-15,6,6,6
1950-05-15,6,6,6
...,...,...,...
2023-08-15,4,4,4
2023-09-15,4,4,4
2023-10-15,3,3,0


In [ ]:
result = ONI_DF.groupby(event_groups).agg(
    min_date=("datetime", "min"),
    max_date=("datetime", "max"),
    count=("ONI", "count")
).rename_axis(None, axis=0)

In [ ]:
result

In [ ]:
ONI_DF

In [ ]:
ONI_DF['group'] = (el_nino_threshold_months != el_nino_threshold_months.shift()).cumsum()

In [ ]:
ONI_DF['group_duration'] = ONI_DF.groupby('group')['ONI'].transform('count')

In [ ]:
ONI_DF_ENSO = ONI_DF[ONI_DF['group_duration'] > 4]


In [ ]:
print(ONI_DF_ENSO)

In [ ]:
%%time
temp_monthly_anomaly = get_monthly_anomaly(temp_chunked_time,temp_monthly_climatology, time_coord_name = 'Time')

In [ ]:
%%time
temp_monthly_anomaly = get_monthly_anomaly(temp_chunked,temp_monthly_climatology, time_coord_name = 'Time')

# What we need : just a surface slice of SST for Indo-Pacific (for example)

In [ ]:
DS_slice = DS.isel(st_ocean=0).sel({'xt_ocean':slice(90,200),'yt_ocean':slice(-50,40)})

In [ ]:
DS_slice.temp.isel(Time=0).plot()

In [ ]:
DS_slice.nbytes/1e9

## this selection is reduced to less than 1.5 GB from the 16TB total

# produce a monthly climatology

In [ ]:
%%time
clim_uv = DS_slice.groupby('Time.month').mean(method="cohorts", engine="flox")

In [ ]:
clim_uv

# This has reduced the result from a 16TB dataset to 2.5 MB so we can easily compute it into our limited memory

In [ ]:
%%time
clim_uv = clim_uv.compute()

# TLDR

## For BRAN2020 (1993 - 2022) - compute climatology of ocean currents
### 8 lines of code and less than a minute of NCI ARE walltime costing 20 cents per hour

In [ ]:
%%time
BRAN2020_catalog = intake.open_esm_datastore(catalog_path+'BRAN2020.json', columns_with_iterables=['variable']) # load catalogue
search_uv_month = BRAN2020_catalog.search(variable=['u','v'],time_period='month') # search and filter data by variables and time period
DS = search_uv_month.to_dask() # load data lazily with Dask
DS_slice = DS.sel(st_ocean= slice(0,300)).sel(xu_ocean=slice(142,160)).sel(yu_ocean=slice(-25,-10)).mean('st_ocean') # slice out reduced XYZ subset required
clim_uv = DS_slice.groupby('Time.month').mean(method="cohorts", engine="flox") ## calculate climatology & chunking rules everything!
speed = np.sqrt(clim_uv.u**2 + clim_uv.v**2) # calculate current speeds
speed.sel(month=8).plot(robust=True) #plot
plt.title('BRAN2020 current speed\n August climatology')

# The End

In [ ]:
client.shutdown()

## Plot current vectors for August

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import mlab, cm, gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
%matplotlib inline 

In [ ]:
%%time
# Define the u and v components of the currents
time_choice = 8
u = clim_uv.u.sel(month=time_choice)
v = clim_uv.v.sel(month=time_choice)
speed = np.sqrt(u**2 + v**2)

In [ ]:
#plot model data
transform = ccrs.PlateCarree()
cmap = 'Spectral_r'
cbar_label='current speed'
plot_data = speed

###
fig = plt.figure(num=None, figsize=(8, 6), dpi=300, facecolor='w', edgecolor='k')
ax = plt.subplot(projection=ccrs.PlateCarree(180))
ax.set_extent([142,160, -25, -10], ccrs.PlateCarree())
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='white'))
ax.coastlines('50m',linewidth=0.5,edgecolor='grey')
plot_data.plot(transform=transform,cmap=cmap,cbar_kwargs={'label': cbar_label,'shrink':0.5},robust=True)

#plot u/v vectors
# Define the x and y coordinates
x = clim_uv.xu_ocean
y = clim_uv.yu_ocean
ax.quiver(x.values,y.values,u.values,v.values,transform=transform, units='x', width=0.01, scale=0.7, headwidth=2,alpha=0.2)
ax.set_title('BRAN2020 1993-2022\ncurrent speed \n August Climatology')